
# 1. OpenAI VLM (GPT-4*) - Basics
This section demonstrates the basic usage of OpenAI's Vision Language Model (VLM) capabilities using GPT-4.
We will use the OpenAI API to analyze an image and provide detailed textual insights.

**Support Material**:
- https://platform.openai.com/docs/guides/text-generation 
- https://platform.openai.com/docs/guides/vision?lang=node
- https://platform.openai.com/docs/guides/text-generation?text-generation-quickstart-example=image 
- https://platform.openai.com/docs/api-reference/chat


In [1]:
import openai
from dotenv import load_dotenv  
import os
import base64
import json
import textwrap

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')


load_dotenv()
#openAIclient = openai.OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "<your OpenAI API key if not set as env var>"))
openAIclient = openai.OpenAI(api_key= os.getenv("OPENAI_API_KEY"))




TEXTMODEL = "gpt-4o-mini" 
IMGMODEL= "gpt-4o-mini" 

# Path to your image
img = "images/street_scene.jpg"

In [2]:
#basic call to gpt4 with prompt and image

completion = openAIclient.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "What's in this image?"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{encode_image(img)}",
                        #"detail": "low"
                    }
                },
            ],
        }
    ],
)


# Wrap the text to a specified width
# Dies extrahiert die Antwort des Modells aus der API-Antwort. completion.choices[0].message gibt die erste Wahl (Antwort) des Modells zurück.
# str(): Wandelt die Antwort in einen String um, damit sie weiterverarbeitet werden kann.
response = str(completion.choices[0].message)
print(textwrap.fill(response, width=120))  # --< Diese Funktion wird verwendet, um den Text umzubrechen, damit er leichter zu lesen ist


ChatCompletionMessage(content="The image depicts a busy urban street scene with various characters and elements:\n\n- In
the foreground, there's a young person sitting on the ground, focused on a device, while another figure appears to be
lying nearby.\n- A flower pot with blooms is positioned next to them.\n- A bench with two people is visible; one is
reading a newspaper while the other is looking at a book.\n- In the background, several vehicles are moving along the
street, and people are crossing the road.\n- A cyclist and a person holding a guitar are also present in the scene,
alongside some pigeons. \n- The surrounding architecture features a mix of historic and modern buildings, contributing
to the lively urban atmosphere. \n\nOverall, the scene captures the hustle and bustle of city life.", refusal=None,
role='assistant', audio=None, function_call=None, tool_calls=None)


messages=[...]: Dies ist die Eingabe, die an das Modell gesendet wird, und sie folgt dem Format eines Dialogs:
role: Gibt die Rolle der Nachricht an. In diesem Fall ist der Benutzer die Quelle der Eingabe mit "role": "user".
content: Dies sind die tatsächlichen Daten, die an das Modell geschickt werden. Es handelt sich um eine Liste von Nachrichten:
Erste Nachricht: Ein einfacher Textprompt: "What's in this image?". Das Modell wird aufgefordert, den Inhalt des Bildes zu beschreiben.
Zweite Nachricht: Ein Bild, das im Base64-Format eingebunden wird. Das Bild wird durch den Base64-String kodiert und als Bild-URL übergeben. Es wird innerhalb des image_url-Feldes übergeben.



# 1.1 Structured Output
Here, we expand upon the VLM example to request structured outputs. This approach allows for extracting 
well-organized information from images in a machine-readable format, such as JSON.

**Support Material**:
- https://platform.openai.com/docs/guides/text-generation?text-generation-quickstart-example=json


In [3]:

def promptLLM(prompt : str = None, sysprompt : str = None,  image : str = None, wantJson : bool = False, returnDict : bool = False):
    returnValue = ""
    messages = [{"role": "system", "content" : sysprompt}]
    modelToUse = TEXTMODEL
    #force it to be a json answer prompt
    #prompt = prompt if not wantJson else returnJSONAnswerPrompt(prompt)
    messages.append({"role": "user", "content": [{ 
        "type" : "text", 
        "text" : prompt 
    }]})
    if image is not None:
        image = f"data:image/jpeg;base64,{image}"
        messages[1]["content"].append({"type": "image_url", "image_url": { "url" : image}})
        modelToUse = IMGMODEL

    if wantJson:
        returnValue = openAIclient.chat.completions.create(
            model=modelToUse,
            #max_tokens= 400,
            response_format={ "type": "json_object" },
            messages=messages,
            temperature=0,
            #n=1,
        )
    else :
        returnValue = openAIclient.chat.completions.create(
            model=modelToUse,
            messages=messages,
            temperature=0,
            #n=1,
        )
    returnValue = returnValue.choices[0].message.content
    if returnDict:
        return json.loads(returnValue)
    return returnValue



# Die Funktion promptLLM() dient dazu, eine Anfrage (Prompt) an ein LLM (Large Language Model) zu senden und entweder eine einfache Antwort oder eine JSON-Antwort zurückzugeben. 
# Sie akzeptiert eine Reihe von Parametern und nutzt die OpenAI-API, um die Anfrage zu verarbeiten. 
# Hier ist eine Erklärung der einzelnen Teile und Parameter der Funktion:
    # prompt (str, optional): Der Text, den der Benutzer an das Modell senden möchte, um eine Antwort zu erhalten. Wenn dieser nicht None ist, wird er als Benutzeranfrage an das Modell gesendet.
    # sysprompt (str, optional): Ein System-Prompt, der dem Modell zusätzliche Anweisungen oder Kontext gibt. Dies könnte verwendet werden, um das Verhalten des Modells zu steuern (z.B. damit es eine detaillierte Antwort gibt oder in einem bestimmten Stil antwortet).
    # image (str, optional): Ein Bild im Base64-codierten Format. Wenn dieses Bild angegeben wird, wird es zusammen mit dem Text-Prompt an das Modell gesendet. Das Bild wird in eine URL umgewandelt und als image_url in der Anfrage hinzugefügt.
    # wantJson (bool, optional): Wenn True, wird die Antwort als JSON-Objekt zurückgegeben. Andernfalls wird die Antwort als normaler Text zurückgegeben.
    # returnDict (bool, optional): Wenn True, wird die Antwort als Python-Wörterbuch (dict) zurückgegeben, nachdem sie aus der JSON-Antwort des Modells extrahiert wurde.

## Funktionsweise:

1. **Systemnachricht (sysprompt):**
   - Es wird eine Nachricht mit der Rolle "system" erstellt, die Kontext oder Anweisungen für das Modell enthält.

2. **Benutzernachricht (prompt):**
   - Diese Nachricht wird unter der Rolle "user" an das Modell gesendet. Sie enthält den Text (und das Bild, falls vorhanden), den das Modell analysieren soll.

3. **Bildverarbeitung:**
   - Falls ein Bild angegeben wird, wird es in ein Base64-codiertes Format umgewandelt und als URL in die Anfrage eingefügt.

4. **Modellauswahl:**
   - Basierend auf dem Vorhandensein eines Bildes wird entweder **TEXTMODEL** oder **IMGMODEL** als Modell ausgewählt.
   - **TEXTMODEL** wird verwendet, wenn nur Text verarbeitet wird, während **IMGMODEL** verwendet wird, wenn ein Bild mit gesendet wird.

5. **API-Anfrage:**
   - Die Anfrage wird an die OpenAI-API gesendet, wobei entweder ein normales Textformat oder ein JSON-Format angefordert wird.

6. **Antwortverarbeitung:**
   - Die Antwort des Modells wird verarbeitet und entweder als Text oder als JSON-Daten zurückgegeben.


In [4]:
#funktionsaufruf (von oben)
output = promptLLM(prompt = "describe the image in detail",sysprompt = "you are a careful observer. the response should be in json format", image = encode_image(img), wantJson=True, returnDict=True)


# prompt: "describe the image in detail" – Dies ist der Text, der an das Modell gesendet wird, um eine detaillierte Beschreibung des Bildes zu erhalten. Der Benutzer fordert das Modell auf, die Szene im Bild genau zu beschreiben.

# sysprompt: "you are a careful observer. the response should be in json format" – Hierbei handelt es sich um den System-Prompt. Er gibt dem Modell den Hinweis, dass es eine sehr sorgfältige Beobachtung der Szene durchführen soll und dass die Antwort im JSON-Format sein soll.

# image: encode_image(img) – Dies ist das Bild, das codiert wurde, um es in einem Format zu übermitteln, das die API verstehen kann. encode_image(img) gibt die Base64-codierte Version des Bildes zurück.

# wantJson: True – Dies gibt an, dass die Antwort im JSON-Format gewünscht wird.

# returnDict: True – Dies bedeutet, dass die Antwort als Python-Wörterbuch (Dictionary) zurückgegeben wird, sodass du die Daten weiterverarbeiten kannst.

In [5]:
output

{'description': {'setting': 'A bustling urban street scene in a city, showcasing a mix of modern and historical architecture.',
  'foreground': {'elements': [{'type': 'bench',
     'details': {'material': 'wood',
      'position': 'sidewalk',
      'occupants': [{'gender': 'female',
        'appearance': {'hair': 'short, blonde',
         'clothing': 'red top, blue jeans',
         'activity': 'reading a magazine'}},
       {'gender': 'male',
        'appearance': {'hair': 'gray',
         'clothing': 'suit',
         'activity': 'reading a newspaper'}}]}},
    {'type': 'child',
     'details': {'gender': 'male',
      'appearance': {'hair': 'short, brown',
       'clothing': 'green jacket, shorts',
       'activity': 'using a smartphone',
       'position': 'sitting on the ground'}}},
    {'type': 'person',
     'details': {'gender': 'male',
      'appearance': {'clothing': 'red hoodie',
       'position': 'lying on the ground'}}},
    {'type': 'pigeons',
     'details': {'quantity': 

In [6]:
output["description"]["foreground"]

{'elements': [{'type': 'bench',
   'details': {'material': 'wood',
    'position': 'sidewalk',
    'occupants': [{'gender': 'female',
      'appearance': {'hair': 'short, blonde',
       'clothing': 'red top, blue jeans',
       'activity': 'reading a magazine'}},
     {'gender': 'male',
      'appearance': {'hair': 'gray',
       'clothing': 'suit',
       'activity': 'reading a newspaper'}}]}},
  {'type': 'child',
   'details': {'gender': 'male',
    'appearance': {'hair': 'short, brown',
     'clothing': 'green jacket, shorts',
     'activity': 'using a smartphone',
     'position': 'sitting on the ground'}}},
  {'type': 'person',
   'details': {'gender': 'male',
    'appearance': {'clothing': 'red hoodie',
     'position': 'lying on the ground'}}},
  {'type': 'pigeons',
   'details': {'quantity': 'multiple', 'activity': 'foraging on the ground'}},
  {'type': 'flower pot',
   'details': {'flowers': 'geraniums', 'position': 'next to the child'}}]}


# JSON Schema for Controlled Structured Outputs
In this section, we define a JSON schema for a more controlled and specific output from the model. 
Using this schema, we can ensure the model adheres to predefined data types and structures while describing images.In this case we will provide an exmaple of json format answer, but ideally 
one could also do it via e.g. pydantic library.

Example: 
```
from typing import List, Literal
from pydantic import BaseModel, Field


class Person(BaseModel):
    position: str = Field(..., description="Position of the person in the environment, e.g., standing, sitting, etc.")
    age: int = Field(..., ge=0, description="Age of the person, must be a non-negative integer.")
    activity: str = Field(..., description="Activity the person is engaged in, e.g., reading, talking, etc.")
    gender: Literal["male", "female", "non-binary", "other", "prefer not to say"] = Field(
        ..., description="Gender of the person"
    )


class ImageExtraction(BaseModel):
    number_of_people: int = Field(..., ge=0, description="The total number of people in the environment.")
    atmosphere: str = Field(..., description="Description of the atmosphere, e.g., calm, lively, etc.")
    hour_of_the_day: int = Field(..., ge=0, le=23, description="The hour of the day in 24-hour format.")
    people: List[Person] = Field(..., description="List of people and their details.")

```

### Änderungen in der `promptLLM`-Methode

1. **JSON-Schema für die Ausgabe:**  
   - Einführung eines JSON-Schemas zur Strukturierung der Modellantwort (z.B. `numberOfPeople`, `atmosphere`, `hourOfTheDay`, `people`).

2. **Strukturierte Antwort:**  
   - Modellantwort muss vordefinierte Felder und Typen wie `age`, `activity`, `gender` enthalten.

3. **Flexible Ausgabe:**  
   - `wantJson` steuert, ob die Antwort als Text oder JSON zurückgegeben wird.
   - Möglichkeit, die Antwort als Dictionary zu erhalten (`returnDict=True`).

4. **Beispiel-JSON-Schema:**  
   - Antwort muss Felder wie `numberOfPeople` und `people` mit spezifischen Datentypen enthalten.


In [7]:
# verbesserte Version der vorherigen promptLLM-Methode
def promptLLM(prompt : str = None, sysprompt : str = None,  image : str = None, wantJson : bool = False, returnDict : bool = False):
    returnValue = ""
    messages = [{"role": "system", "content" : sysprompt}]
    modelToUse = TEXTMODEL
    #force it to be a json answer prompt
    #prompt = prompt if not wantJson else returnJSONAnswerPrompt(prompt)
    messages.append({"role": "user", "content": [{ 
        "type" : "text", 
        "text" : prompt 
    }]})
    if image is not None:
        image = f"data:image/jpeg;base64,{image}"
        messages[1]["content"].append({"type": "image_url", "image_url": { "url" : image}})
        modelToUse = IMGMODEL

    if wantJson:
        returnValue = openAIclient.chat.completions.create(
            model=modelToUse,
            #max_tokens= 400,
            response_format={
                "type": "json_schema",
                "json_schema": {
                    "name": "img_extract",
                    "schema": {
                    "type": "object",
                    "properties": {
                        "numberOfPeople": {
                        "type": "integer",
                        "description": "The total number of people in the environment",
                        "minimum": 0
                        },
                        "atmosphere": {
                        "type": "string",
                        "description": "Description of the atmosphere, e.g., calm, lively, etc."
                        },
                        "hourOfTheDay": {
                        "type": "integer",
                        "description": "The hour of the day in 24-hour format",
                        "minimum": 0,
                        "maximum": 23
                        },
                        "people": {
                        "type": "array",
                        "description": "List of people and their details",
                        "items": {
                            "type": "object",
                            "properties": {
                            "position": {
                                "type": "string",
                                "description": "Position of the person in the environment, e.g., standing, sitting, etc."
                            },
                            "age": {
                                "type": "integer",
                                "description": "Age of the person",
                                "minimum": 0
                            },
                            "activity": {
                                "type": "string",
                                "description": "Activity the person is engaged in, e.g., reading, talking, etc."
                            },
                            "gender": {
                                "type": "string",
                                "description": "Gender of the person",
                                "enum": ["male", "female", "non-binary", "other", "prefer not to say"]
                            }
                            },
                            "required": ["position", "age", "activity", "gender"]
                        }
                        }
                    },
                    "required": ["numberOfPeople", "atmosphere", "hourOfTheDay", "people"]
                    }}},
            messages=messages,
            temperature=0,
            #n=1,
        )
    else :
        returnValue = openAIclient.chat.completions.create(
            model=modelToUse,
            messages=messages,
            temperature=0,
            #n=1,
        )
    returnValue = returnValue.choices[0].message.content
    if returnDict:
        return json.loads(returnValue)
    return returnValue

In [8]:
output_image_analysis = promptLLM(prompt = "describe the image in detail",sysprompt = "you are a careful observer. the response should be in json format", image = encode_image(img), wantJson=True, returnDict=True)

# Das resultierende JSON gibt eine detaillierte Beschreibung des Bildes zurück, einschließlich der Anzahl der Personen, ihrer Aktivitäten und der allgemeinen Atmosphäre.

In [9]:
#alert service prompt  --> das Modell darauf vorzubereiten, eine Analyse der Szene in Bezug auf potenziell gefährdete Personen zu liefern
                        #  potenziell gefährliche Situationen zu erkenne

alert_sys_prompt = " you are an experienced first aid paramedical"
alert_prompt= """Extract from the following scene analysis give to you in json format, 
if anyone might be in danger and if the Child Hospital or normal Hospital should be alerted. 
Give the a concise answer
The situation is given to you from this object: """ + str(output_image_analysis)


In [10]:
promptLLM(prompt = alert_prompt, sysprompt= alert_sys_prompt) 


#  alert_prompt: Dies ist der eigentliche Text (Prompt), der beschreibt, was das Modell tun soll. In diesem Fall fordert der Prompt das Modell auf, potenziell gefährliche Situationen aus einer JSON-Darstellung einer Bildanalyse zu extrahieren und gegebenenfalls eine Empfehlung zu geben, ob das Krankenhaus oder ein Kinderkrankenhaus benachrichtigt werden sollte.

# alert_sys_prompt: Dies ist der System-Prompt, der dem Modell eine Rolle zuweist, um die Aufgabe auszuführen. In diesem Fall wird das Modell als ein erfahrener paramedizinischer Notfallsanitäter eingesetzt, der in der Lage ist, medizinische Notfälle korrekt zu erkennen und zu bewerten.

'In this scene, the 15-year-old male who is lying down and appears to be unconscious or resting may be in danger. It is advisable to alert a normal hospital for potential medical assistance, as the situation does not specifically indicate a pediatric emergency that would require a Child Hospital.'

In [11]:
promptLLM(prompt = "Considering the image analysis given" +str(output_image_analysis)+ "give me back the coordinates of the 16-years old. If these are not available, infer them form the pic", sysprompt= alert_sys_prompt) 

'Based on the provided image analysis, there is no mention of a 16-year-old individual. The closest ages mentioned are 15 and 20. Since there is no specific data for a 16-year-old, I cannot provide coordinates for that age group.\n\nIf you need assistance with a different age group or any other information, please let me know!'

verlangt vom Modell, dass es die Bildanalyse berücksichtigt und die Koordinaten eines 16-jährigen Individuums zurückgibt. Falls keine expliziten Daten für eine 16-jährige Person vorliegen, soll das Modell versuchen, die Koordinaten basierend auf dem Bild zu schätzen.

Die Antwort des Modells lautet:

"Basierend auf der bereitgestellten Bildanalyse gibt es keine Erwähnung eines 16-jährigen Individuums. Die nächstgelegenen Altersangaben sind 15 und 20. Da keine spezifischen Daten für einen 16-Jährigen vorliegen, kann ich keine Koordinaten für diese Altersgruppe bereitstellen."

In [13]:
promptLLM(prompt =  "Detect if there is a person who is under 18 years old on the floor and reutrn its coordinates as a list in the format '[ymin,xmin, ymax, xmax]'. Just output the list.", sysprompt= alert_sys_prompt, image = encode_image(img)) 

'[400, 600, 500, 700]'


# 2. Google VLM (Gemini)
This section demonstrates the use of Google's Vision Language Model, Gemini. 
We explore basic text generation as well as its ability to analyze images and provide relevant outputs.

**Support Material**:
- https://colab.research.google.com/drive/1eDvf_Ky9jLOZFShgHrm4GI-wkAaQnue6?usp=sharing


In [14]:
%matplotlib inline
import os
from dotenv import load_dotenv  
import google.generativeai as genai
from PIL import Image

load_dotenv()
#genai.configure(api_key=os.environ.get("GEMINI_API_KEY"))
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

# pip install google-generativeai


/home/vscode/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
model = genai.GenerativeModel("gemini-1.5-flash")
response = model.generate_content("Explain how AI works")
print(response.text)


Artificial intelligence (AI) is a broad field encompassing various techniques that enable computers to mimic human intelligence.  There's no single "how it works" answer, as different AI approaches utilize different methods. However, we can break down some core concepts:

**1. Data is King:**  AI systems learn from data.  The more relevant and high-quality data you feed an AI, the better it performs. This data can be anything from images and text to sensor readings and financial transactions.

**2. Algorithms are the Engine:** Algorithms are sets of rules and statistical techniques that AI systems use to process data and learn patterns.  These algorithms are designed to:

* **Identify patterns:** Find relationships and correlations within the data.
* **Make predictions:** Use learned patterns to forecast future outcomes or classify new data.
* **Learn and adapt:** Modify their behavior based on new data and feedback.

**3. Key AI Techniques:**  Several prominent techniques drive AI sys

### Model Setup:
- `genai.GenerativeModel("gemini-1.5-flash")` initializes the Gemini model (version 1.5) for generating content.
- The `generate_content` method is used to send a prompt to the model, in this case, asking for an explanation of AI.

### Response:
- The model processes the input prompt and generates a response based on its training, returning the output in `response.text`.

### Output:
- The text generated would then provide an explanation of how AI works, which may describe AI's ability to learn from data, use algorithms, and make predictions.


In [16]:
im = Image.open(img)

genai.configure(api_key=os.environ.get("GEMINI_API_KEY"))
model = genai.GenerativeModel("gemini-1.5-pro")

response = model.generate_content([
    im,
    (
        "Detect if there is a person who is under 18 years old on the floor and reutrn its coordinates as a list in the format '[ymin,xmin, ymax, xmax]'. Just output the list.\n "
    ),
])
response.resolve()
print(response.text)

[699, 326, 959, 627]


### Breakdown of Code for Image Analysis Using Google Gemini Model

1. **Image Loading**:
   The image is loaded using the Python Imaging Library (PIL) with the function `Image.open(img)`. This reads the image and prepares it for processing by the model.

2. **Model Configuration**:
   The Gemini model is configured with an API key from the environment variable `GEMINI_API_KEY`, ensuring secure access to the model's services.

3. **Model Initialization**:
   The code uses the `gemini-1.5-pro` version, which is optimized for handling complex tasks like image analysis and object detection. This model is capable of generating content based on the prompts provided.

4. **Prompt Setup**:
   The `generate_content()` method is called, sending both the image and a prompt to the model. The prompt specifically instructs the model to detect if there is a person under 18 years old lying on the floor. It is asked to return the coordinates of the detected individual in the format `[ymin, xmin, ymax, xmax]`.

5. **Response Handling**:
   After the model processes the input, the `response.resolve()` method ensures that the generated content is correctly processed. The result is then displayed with `response.text`, which contains the coordinates of the detected person, if any.

This process can be used to automate the identification of specific individuals in images, particularly useful for tasks such as monitoring or emergency response scenarios.


Gemini can be used to predict bounding boxes based on free form text queries.
The model can be prompted to return the boxes in a variety of different formats (dictionary, list, etc). This of course migh need to be parsed. 
Check: https://colab.research.google.com/drive/1eDvf_Ky9jLOZFShgHrm4GI-wkAaQnue6?usp=sharing#scrollTo=WFLDgSztv77H
